## Configuracion

In [0]:
import time
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import (
    EndpointCoreConfigInput,
    ServedModelInput,
    ServedModelInputWorkloadSize,
    EndpointStateReady
)
from config_loader import Config

paths = Config.get_paths()
params = Config.get_params()

fe_model_name = f"{paths['catalog']}.{paths['schemas']['feature_store']}.feature_engineering"
fe_alias = params['model_params']['alias']  

fe_endpoint_name = "fe_endpoint"
workload_size = ServedModelInputWorkloadSize[params['serving_params']['workload_size']]
scale_to_zero = params['serving_params']['scale_to_zero']

## VERSION DEL MODELO

In [0]:
w = WorkspaceClient()

try:
    model_details = w.model_versions.get_by_alias(fe_model_name, alias=fe_alias)
    version_to_deploy = model_details.version
    print(f"📦 Detectada versión {version_to_deploy} para el alias '{fe_alias}'")
except Exception as e:
    print(f"❌ Error: No se encontró el alias '{fe_alias}' en el modelo {fe_model_name}")
    raise e


## CONFIGURACION DEL ENDPOINT

In [0]:
serving_config = EndpointCoreConfigInput(
    served_models=[
        ServedModelInput(
            model_name=fe_model_name,
            model_version=version_to_deploy,
            workload_size=workload_size,
            scale_to_zero_enabled=scale_to_zero
        )
    ]
)

## CREAR O ACTUALIZAR ENDPOINT

In [0]:
existing_endpoints = [e.name for e in w.serving_endpoints.list()]

if fe_endpoint_name in existing_endpoints:
    print(f"🔄 Actualizando endpoint '{fe_endpoint_name}' a la versión {version_to_deploy}")
    w.serving_endpoints.update_config(name=fe_endpoint_name, served_models=serving_config.served_models)
else:
    print(f"🚀 Creando nuevo endpoint '{fe_endpoint_name}'")
    w.serving_endpoints.create(name=fe_endpoint_name, config=serving_config)

print("⏳ Esperando a que el endpoint esté listo (esto puede tardar unos minutos)...")

while True:
    endpoint = w.serving_endpoints.get(name=fe_endpoint_name)
    status = endpoint.state.ready

    print(f"   Estado actual: {status}")

    if status == EndpointStateReady.READY:
        print(f"✅ ¡Éxito! Endpoint '{fe_endpoint_name}' activo.")
        break
    elif status == EndpointStateReady.NOT_READY:
        print("⚠️ El endpoint reporta estado NOT_READY. Revisá los logs en el UI.")
        time.sleep(20)